In [ ]:
import pandas as pd

df = pd.read_csv('../data/trajectories.csv')

trayectories_df = pd.DataFrame(df)

# Delecciona disinct dates
dates = trayectories_df['date'].unique()

dates_df = pd.DataFrame(dates, columns=['date'])

import matplotlib.pyplot as plt

plt.scatter(trayectories_df['longitude'], trayectories_df['latitude'], s=0.1)


In [ ]:

# Selecciona netCDF de acuerdo a las fechas del dataframe
import os
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

def select_netCDf(date):
    path = '../data/sst/2014/01'
    # Cambia formato de yyyy-mm-dd a yyyymmdd
    date = date.replace('-', '')
    filename = f"{path}/{date}120000-ESACCI-L4_GHRSST-SSTdepth-OSTIA-GLOB_CDR2.1-v02.0-fv01.0_clipped.nc"
    # Plot netCDF
    nc_file = nc.Dataset(filename)
    print(nc_file.variables.keys())
    # Get lat and lon
    lat = nc_file.variables['lat'][:]
    lon = nc_file.variables['lon'][:]
    # Get SST
    sst = nc_file.variables['analysed_sst'][0][:,:]
    sst = sst - 273.15
    # Close netCDF
    nc_file.close()
    return lon, lat, sst


In [ ]:
def plot_netCDF(date):
    lon, lat, sst = select_netCDf(date)
    # Plot netCDF
    #plt.pcolor(lon, lat, sst)
    #plt.title(f"Sea Surface Temperature {date}")

fecha = dates_df['date'][3]  
plot_netCDF(fecha)

In [ ]:
import matplotlib.pyplot as plt

# Grafica puntos de una fecha en particular
def get_points(date):
    points = trayectories_df[trayectories_df['date'] == date]
    return points

In [ ]:
import numpy as np
# Obten temperaturas de netCDF a partir de coordenadas de puntos

lon, lat, sst = select_netCDf(fecha)
puntos = pd.DataFrame(get_points(fecha))

# Plot netCDF
plt.pcolor(lon, lat, sst, cmap='jet')
plt.colorbar()
plt.scatter(puntos['longitude'], puntos['latitude'], s=0.1, c='DarkBlue')
# Zoom to points
plt.xlim(puntos['longitude'].min(), puntos['longitude'].max())
plt.ylim(puntos['latitude'].min(), puntos['latitude'].max())


# Obtener temperatura de netCDF a partir de coordenadas de puntos
def get_sst(longitude, latitude, sst, puntos):
    id = []
    sst_points = []
    for i in range(len(puntos)):
        # Obten coordenadas de punto
        lon_point = puntos['longitude'].iloc[i]
        lat_point = puntos['latitude'].iloc[i]
        # Obten indice de coordenadas en netCDF
        lon_index = (abs(longitude - lon_point)).argmin()
        lat_index = (abs(latitude - lat_point)).argmin()
        # Obten temperatura de netCDF
        sst_point = sst[lat_index,lon_index]
        sst_points.append(sst_point)
    puntos['sst'] = sst_points
    return puntos

sst_onsite = get_sst(lon, lat, sst, puntos)
sst_onsite